#Importanto as Bibliotecas

In [19]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

#Base de Textos

In [3]:
texts = ['Goku is a hero in the Dragon Ball since 1989! Goku saved the earth so many times.',
         'The 7 Dragon balls can make wishes come true! Each ball contains his own dragon.',
         'If the wishes are superfluous, the dragon balls will become dark.' ,
         'Seiya is a bronze knight and is one of the main Knights of the Zodiac. He saved Athena several times.',
         "A knight of the zodiac wear a bronze, silver or a gold cloth to protect Athena.",
         'Saint Seiya: Knights of the Zodiac is a Japanese manga in which mystical warriors called the Saints fight wearing sacred cloths.']
classes = ['Dragon Ball', 'Dragon Ball', 'Dragon Ball', 'Cav. Zod.', 'Cav. Zod.', 'Cav. Zod.']

df = pd.DataFrame({'texts': texts, 'classes': classes})

In [4]:
df

,texts,classes
0,Goku is a hero in the Dragon Ball since 1989! ...,Dragon Ball
1,The 7 Dragon balls can make wishes come true! ...,Dragon Ball
2,"If the wishes are superfluous, the dragon ball...",Dragon Ball
3,Seiya is a bronze knight and is one of the mai...,Cav. Zod.
4,"A knight of the zodiac wear a bronze, silver o...",Cav. Zod.
5,Saint Seiya: Knights of the Zodiac is a Japane...,Cav. Zod.


# Vetorizando os documentos de treinamento

In [20]:
"""Criando o objeto vetorizador (convertendo para caixa baixar, removendo as stopwords especificadas em uma lista)
   e apenas mantendo os termos que ocorreram em dois ou mais documentos"""
vetorizador = CountVectorizer(lowercase=True,
                              stop_words=['is', 'are', 'a', 'or', 'to', 'in', 'the', 'so', 'since', 'many', 'of'], 
                              min_df=2,
                              dtype=np.int16
                              )

In [21]:
#Obtendo o vocabulário
vetorizador.fit(df['texts'])

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int16'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=2,
                ngram_range=(1, 1), preprocessor=None,
                stop_words=['is', 'are', 'a', 'or', 'to', 'in', 'the', 'so',
                            'since', 'many', 'of'],
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [17]:
vetorizador.vocabulary_

{'athena': 0,
 'ball': 1,
 'balls': 2,
 'bronze': 3,
 'dragon': 4,
 'knight': 5,
 'knights': 6,
 'saved': 7,
 'seiya': 8,
 'times': 9,
 'wishes': 10,
 'zodiac': 11}

In [18]:
#Gerando a representação estruturada a partir do vocabulário ajustado
representacao = vetorizador.transform(df['texts'])

In [22]:
#A representação retornada é uma matriz em um formato esparso
representacao

<6x12 sparse matrix of type '<class 'numpy.int64'>'
	with 26 stored elements in Compressed Sparse Row format>

In [23]:
"""Para visualizá-la, vamos utilizar o método toarray() para retornar a matriz em formato
   não esparso além de pegar o vocabulário pela order dos índices"""

#Obtendo os termos ordenados de acordo com o seu índice no vocabulário
colunas = [item[0] for item in sorted(vetorizador.vocabulary_.items(), key = lambda x: x[1])]

#Gerando o DataFrame com o dados
df_repr = pd.DataFrame(representacao.toarray(), columns=colunas)

In [24]:
df_repr

,athena,ball,balls,bronze,dragon,knight,knights,saved,seiya,times,wishes,zodiac
0,0,1,0,0,1,0,0,1,0,1,0,0
1,0,1,1,0,2,0,0,0,0,0,1,0
2,0,0,1,0,1,0,0,0,0,0,1,0
3,1,0,0,1,0,1,1,1,1,1,0,1
4,1,0,0,1,0,1,0,0,0,0,0,1
5,0,0,0,0,0,0,1,0,1,0,0,1


#Treinando um modelo de classificação

In [29]:
#Importanto um modelo de classificação. Neste caso foi importando o MultinomialNB
from sklearn.naive_bayes import MultinomialNB

In [30]:
#Criando um objeto com o modelo 
modelo = MultinomialNB()

In [31]:
#Ajustando o modelo aos dados de treinamento (etapa de aprendizado)
modelo.fit(representacao, classes)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

# Predizendo novos exemplos

In [36]:
# Criando exemplos de teste
texts_test = ['Goku will have to save the earth again in Dragon Ball', 
              'Seiya is loyal to Athena']

In [37]:
#Vetorizando os exemplos de teste com o vetorizador ajustado nos dados de treinamento
representacao_testes = vetorizador.transform(texts_test)

In [38]:
# Fazendo a predição com o modelo de classificação
modelo.predict(representacao_testes)

array(['Dragon Ball', 'Cav. Zod.'], dtype='<U11')